In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup


import pymongo
import dns
import json

import os
from dotenv import load_dotenv

In [16]:
load_dotenv('./.env')

True

In [6]:
def insert_into_db(collection_name,object_to_add):
    client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
    db = client.stats
    db[collection_name].insert_one(object_to_add)

In [27]:
def scrape_stats(start_year : int, end_year : int):
    
#     Initialize mongodb connection
    client = pymongo.MongoClient(os.getenv("MONGODB_URI_ADVANCED_STATS"))
    db = client.stats

    
    for year in range(start_year, end_year + 1):
        year = str(year)
        
        
        url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        
        
        for i, row in enumerate(soup.find("div", id = "div_per_game_stats").find('tbody').find_all('tr')):
            
            stat_dictionary = {}
            
            if "thead" in row.get('class'):                
                continue
            
            try:
                for ind, col in enumerate(row):
                    if ind != 0:
                        stat_dictionary[col.get('data-stat')] = col.text
                    
            except AttributeError:
                pass
            
            
            
            advanced_stats_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
            advanced_source = requests.get(advanced_stats_url).text
            soup_advanced = BeautifulSoup(advanced_source, 'lxml')
            
            
            advanced_col = soup_advanced.find("div", id = "div_advanced_stats").find('tbody').find_all('tr')[i]
            
            try:
                for ind, col in enumerate(advanced_col):
                    if ind != 0 and ind != 1:
                        stat_dictionary[col.get('data-stat')] = col.text
            
            except AttributeError:
                pass
            
            
            db[year].insert_one(stat_dictionary)
#             insert_into_db(year, stat_dictionary)
            
            
        
            
    


In [ ]:
scrape_stats(2019,2022)

In [1]:
# Web scrapper for getting playerID and general stats

In [2]:
def get_player_data():
    

    url = "https://stats.nba.com/stats/playerindex?College=&Country=&DraftPick=&DraftRound=&DraftYear=&Height=&Historical=1&LeagueID=00&Season=2022-23&SeasonType=Regular%20Season&TeamID=0&Weight="
    
    payload={}
    headers = {
      'Accept': '*/*',
      'Accept-Language': 'en-CA,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
      'Connection': 'keep-alive',
      'Origin': 'https://www.nba.com',
      'Referer': 'https://www.nba.com/',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-site',
      'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Linux"',
      'Cookie': 'ak_bmsc=22E0E68CE2DD8F7B78773537CCDA3527~000000000000000000000000000000~YAAQrw/QF6i20gOCAQAAZ+XOExB7o9knL/op+4Ke/sNJK9NC4PD/Tne2C6QxXSEB2QGCOvW9hRNCpoybhX+sRqacKeEuM4VzeIEWLNM/NDT/nsAaNJfw2s5BbGXd9RHmO/6ZzSDBxjAtLRYhicY/JIWK3+w3TdJw0K/CQX5X0XX9rstSDPy1E8MMduQFTK5rYc4MG7UNHoI9jN/F4USya0WhqFOmx2LYELR9LY4zTyeSQ0O4whTyub7fpOTJ89KF1cmOjtQzFK13Kpj6A9miDqnK5hLpECtO7u/eKNpT40FOjw9VCMCMLVcNZ8USDEz9bjq/HzcuLL+zzmXcAzFTyOSf+2D3uxcoDjHUXllbr6DWJT7NWoB2iY8='
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    
    return response

def convert_to_json(response):
    return response.json()


In [18]:
def insert_into_db(db_name, collection_name, object_to_add):
    client = pymongo.MongoClient(os.getenv("MONGODB_URI_GENERAL_STATS"))
    db = client[db_name]
    db[collection_name].insert_one(object_to_add)



    
def scrape_player_data():
    response = get_player_data()
    player_data = convert_to_json(response)
    
    data_object = {}
    
    for index, data in enumerate(player_data['resultSets'][0]['rowSet']):
        
        if (index >= 2917):
            for index, header in enumerate(player_data['resultSets'][0]['headers']):

                data_object[header] = data[index]

            insert_into_db('player-general-stats','player-data', data_object)
            data_object = {}
    
    print('Finished')

In [ ]:
scrape_player_data() 